In [ ]:
# Installing Required Packages and Dependencies
!pip install - -upgrade llama - index llama - index - embeddings - huggingface llama - index - llms - huggingface transformers torch accelerate bitsandbytes


In [ ]:
# Importing Required Libraries
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
import torch
import os
import pandas as pd


In [ ]:
# Setting Up the Models and Embeddings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    context_window=2048,
    max_new_tokens=256,
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "load_in_4bit": True}
)


In [ ]:
# Loading Data from CSV
csv_file_path = "movie_recommendations_with_names.csv"
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
    print(f"Current working directory: {os.getcwd()}")
except Exception as e:
    print(f"An unexpected error occurred while reading the CSV: {e}")


In [ ]:
# Document Creation and Indexing
movies_data = [
    Document(text=f"MovieID: {row['movie_id']}, Title: {row['title']}, Genre: {row['genre']}, Rating: {row['rating']}",
             metadata={"movie_id": row['movie_id'], "title": row['title'], "genre": row['genre'], "rating": row['rating']})
    for index, row in df.iterrows()
]
index = VectorStoreIndex.from_documents(movies_data)
query_engine = index.as_query_engine()


In [ ]:
# User Interaction Loop
print("Welcome to MovieRecBot! ")
print("How can I assist you in finding your next movie to watch? (Type 'exit' to end)")
while True:
    query_string = input("\nWhat type of movie are you in the mood for?\n ")
    if query_string.lower() == 'exit':
        print("Thank you for using MovieRecs! Enjoy your movie and have a great day!\n")
        break


In [ ]:
# Querying and Formatting Recommendations
response = query_engine.query(
    f"List titles and genres of movies with the genre {query_string} from the provided data. Provide at least 3 recommendations if available.")

response_lines = str(response).split('\n')

filtered_recommendations = [
    line for line in response_lines
    if line.strip() and "Note: The query is not specific to the data provided" not in line
]
formatted_recommendations = []
for rec in filtered_recommendations:
    if "Title:" in rec:
        try:
            title_part = rec.split("Title:")[1].split(",")[0].strip()
            formatted_recommendations.append(f"- {title_part}")
        except:
            formatted_recommendations.append(f"- {rec.strip()}")
    elif rec.strip():
        formatted_recommendations.append(f"- {rec.strip()}")

limited_formatted_recommendations = formatted_recommendations[:5]


In [ ]:
# Display the Result
if limited_formatted_recommendations:
    final_response = "\n".join(limited_formatted_recommendations)
else:
    final_response = "Sorry, I couldn't find any movies of that genre in my list."
if limited_formatted_recommendations:
    final_response = "\n".join(limited_formatted_recommendations)
else:
    final_response = "Sorry, I couldn't find any movies of that genre in my list."

print(f"\nMovieRecs Assistant:\n{final_response}")
